### Итерационные методы (простой итерации, Зейделя, верхней релаксации) решения СЛАУ.



Дана система $Ax=b$


$ A = \begin{pmatrix} 8.673134 & 1.041039 & -2.677712 \\ 1.041039 & 6.586211 & 0.62301 \\ -2.677712 & 0.623016 & 5.225935
 \end{pmatrix}$

 $ b = \begin{pmatrix} -1.289879 \\ 4.020225 \\ 5.269671 \end{pmatrix} $

#### Найти решение методом Гаусса

In [49]:
# импортируем необходимые модули
import matrix_funs as mf
import numpy as np
from numpy import linalg as LA
import math
import copy


name = "/home/tatyana/PycharmProjects/Computationals_2/matrix_try.txt"

a_ext = np.loadtxt(name)
n, m = a_ext.shape
a = a_ext[:, 0:m-1]
b = a_ext[:,m-1 ]


x = mf.Gauss(a, b)
print(x)

[0.1 0.5 1. ]


#### Привести систему к виду $x = H_Dx + g_D$ и найти $||H_D||_\infty$

In [50]:
E_matrix = np.eye(n)
D_matrix = np.zeros((n, n))

for i in range(n):
    D_matrix[i][i] = a[i][i]

D_inv = LA.inv(D_matrix)        # матрица обратная к D
C = D_inv.dot(a)
H_D = E_matrix - C
g_D = D_inv.dot(b)      # столбец g_D
H_norm = mf.matrix_norm(H_D)        # норма матрицы H_D

print("Матрица H_D:",'\n', H_D)
print("Столбец g_D:",'\n',g_D)
print("Норма матрицы H_D: ", H_norm)

Матрица H_D: 
 [[ 0.         -0.12003031  0.30873638]
 [-0.15806341  0.         -0.09459399]
 [ 0.51238907 -0.11921618  0.        ]]
Столбец g_D: 
 [-0.14872121  0.61040027  1.00836903]
Норма матрицы H_D:  0.6316052534139824


#### Найти априорную оценку $k$ т.ч. $||x^k-x^*||_\infty<\epsilon$, $\epsilon = 0.00001$
$||x^k - x^*||\le||H||^k(||x^o|| + \frac{||g||}{1 - ||H||})$
Для достижения необходимой точности $\epsilon$ нужно выполнить число итераций $k$, которое является ближайшим
натуральным числом таким, что

$k \ge \frac{\ln(\frac{\epsilon\cdot(1 - ||H||}{||x^0|| - ||H||\cdot||x^0|| + ||g||})}
{\ln(||H||)} $

Нулевое приближение - нулевой вектор.

Тогда неравенство принимает вид:

$k \ge \frac{\ln(\frac{\epsilon\cdot(1 - ||H||)}{||g||})}{\ln(||H||)}$


In [51]:
# точность вычислений
eps = 10**(-5)

# норма столбца g_D
g_norm = max(np.abs(g_D))

# вычисляем правую часть неравенства
val_1 = np.log(eps*(1-H_norm)/g_norm)
val_2 = np.log(H_norm)
val_est =val_1/val_2

k_apr = math.ceil(val_est)
print("априорная оценка k: ", k_apr)

априорная оценка k:  28


#### Метод простейших итераций

In [52]:
xx = LA.solve(a, b)

x_k = np.zeros(n)
x_k_1 = g_D
k = 1
while max(np.abs(x_k_1 - x)) > eps and k != k_apr :
    x_k = copy.copy(x_k_1)
    vall = H_D.dot(x_k_1)
    x_k_1 = vall + g_D
    k += 1

print("Получили число итераций: ", k)
print("Приближенное решение: ", x_k_1)
print("Точное решение: ", xx)
print("Фактическая погрешность: ", np.abs(x - x_k_1))

apostr_est = H_norm/(1 - H_norm)*max(np.abs(x_k_1-x_k))     # апостериорная оценка
print("Апостериорная оценка: ", apostr_est)

apr_est = (H_norm**k)/(1 - H_norm)*g_norm       # априорная оценка
print("Априорная оценка", apr_est)

# уточним по Люстернику
w, v = LA.eig(H_D)      # собственные числа и векторы матрицы H
rho_H = max(abs(w))     #спектральный радиус матрицы H

x_Lust = x_k + 1/(1 - rho_H)*(x_k_1-x_k)
print("Уточненное решение по Люстернику: ", x_Lust)

print("Фактическая погрешность по Люстернику: ", np.abs(x - x_Lust))

Получили число итераций:  14
Приближенное решение:  [0.09999504 0.50000349 0.99999154]
Точное решение:  [0.09999996 0.49999996 0.99999983]
Фактическая погрешность:  [4.96163858e-06 3.49476151e-06 8.46274522e-06]
Апостериорная оценка:  1.6414682719844036e-05
Априорная оценка 0.004400949047236786
Уточненное решение по Люстернику:  [0.10000327 0.50000029 0.99999569]
Фактическая погрешность по Люстернику:  [3.26623974e-06 2.86550807e-07 4.30733447e-06]


#### Метод Зейделя, $\epsilon = 0.00001$

$x^{k+1} = (E - H_L)^{-1}\cdot H_R\cdot x^k + (E - H_L)^{-1}\cdot g$

In [53]:
# разложить матрицу H = H_L + H_R
# матрциа H = H_D

H = H_D
H_L = np.zeros((n, n))
H_R = np.zeros((n, n))

for i in range(1, n):
    j = 0
    while j != i:
        H_L[i][j] = H[i][j]
        j += 1

for i in range(0, n):
    j = n-1
    while j >= i:
        H_R[i][j] = H[i][j]
        j -= 1

m_1 = LA.inv(E_matrix - H_L)       # матрица (E - H_L)^(-1)
m_2 = m_1.dot(H_R)

x_k = np.zeros(n)
x_k_1 = m_1.dot(g_D)
k = 1
while max(np.abs(x_k_1 - x_k)) > eps:
    x_k = copy.copy(x_k_1)
    x_k_1 = m_2.dot(x_k) + m_1.dot(g_D)
    k += 1



print("Решение, полученное методом Зейделя: ", x_k_1)
print("Фактическая погрешность: ", np.abs(x - x_k_1))
print("Потребовавшееся количество операций: ", k)

Решение, полученное методом Зейделя:  [0.09999844 0.50000057 0.99999898]
Фактическая погрешность:  [1.55591576e-06 5.65984009e-07 1.01779134e-06]
Потребовавшееся количество операций:  9


#### Метод верхней релаксации

In [54]:
q = 2/(1 + math.sqrt(1 - rho_H**2))         # оптимальное q

x_k = np.zeros(n)
part_val_1 = LA.inv(E_matrix - q*H_L)
x_k_1 = part_val_1.dot(q*g_D)
k = 1

while max(np.abs(x_k_1 - x)) > eps and k!=28 :
    x_k = copy.copy(x_k_1)
    part_val = H_R.dot(x_k) - x_k + g_D
    x_k_1 = part_val_1.dot(x_k + q*part_val)
    k += 1
print("Решение методом Зейделя: ", x_k_1)
print("Число итераций", k)
print("Фактическая погрешность: ", np.abs(x_k_1 - x))

Решение методом Зейделя:  [0.09999524 0.5000011  0.99999801]
Число итераций 6
Фактическая погрешность:  [4.76073963e-06 1.10347158e-06 1.98888296e-06]
